In [2]:
import cv2
import dlib
import numpy as np
import math 

def rect_contain(point):
    if point[0]<0:
        return 0
    elif point[1]<0:
        return 0
    elif point[0]>200:
        return 0
    elif point[1]>200:
       return 0
    else:
       return 1
image=cv2.imread('img.jpeg')
print image.shape
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")
image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

detections = detector(image, 1) #Detect the faces in the image
k=9
p1=[]
p2=[]
for k,d in enumerate(detections): #For each detected face
 print k
 shape = predictor(image, d)
  #There are 68 landmark points on each face
                #For each point, draw a red circle with thickness2 on the original frame
 a=(shape.part(36).x,shape.part(36).y)

 b=(shape.part(45).x,shape.part(45).y)
 p1=[a,b]

  #cv2.circle(image, (p1[i], p2[i]), 2, (0,0,255), -1)

p2=[((int(0.3*200),66)),(int(0.7*200),66)]
s60 = math.sin(60*math.pi/180);
c60 = math.cos(60*math.pi/180);  
inPts = np.copy(p1).tolist();
outPts = np.copy(p2).tolist();
xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
inPts.append([np.int(xin), np.int(yin)]);
xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
outPts.append([np.int(xout), np.int(yout)]);
tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
img2 = cv2.warpAffine(image, tform, (200,200));   
cv2.circle(image,(int(0.3*200),66),4,(0,255,0),-1)
cv2.circle(image,(int(0.7*200),66),4,(0,255,0),-1)
#cv2.circle(image,p1[0],4,(0,255,0),-1)

#cv2.circle(img2,p1[0],4,(0,255,0),-1)

cv2.circle(img2,(int(0.3*200),66),4,(0,255,0),-1)
cv2.circle(img2,(int(0.7*200),66),4,(0,255,0),-1)
print tform
point=[]
for i in range(1,68): 
  point.append((shape.part(i).x,shape.part(i).y))
point=np.array(point)
print point.shape
points2 = np.reshape(point, (67,1,2)); 

points = cv2.transform(points2, tform);
points = np.int64(np.reshape(points, (67, 2)));
boundry=np.array([(0,0),(199,0),(0,199),(199,199),(199,0),(0,100),(199,100),(100,199)])
points=np.append(points,boundry,axis=0)

delaunay=[]
subdiv=cv2.Subdiv2D((0,0,200,200));
for ix in points:
    subdiv.insert((ix[0],ix[1]));
    
triangle=subdiv.getTriangleList()


for t in triangle:
    pt1=(t[0],t[1])
    pt2=(t[2],t[3])
    pt3=(t[4],t[5])
    if rect_contain(pt1) and rect_contain(pt2) and rect_contain( pt3) :
         
            cv2.line(img2, pt1, pt2, (255,255,0), 1)
            cv2.line(img2, pt2, pt3, (255,255,0), 1)
            cv2.line(img2, pt3, pt1, (255,255,0), 1)
 
            delaunay.append((pt1,pt2,pt3))
print len(delaunay)
del1=np.array(delaunay[0])

r1 = cv2.boundingRect(np.array(delaunay[0]))
r1=np.array(r1)
print r1,r1[0]

#for i in boundry.tolist():
#  cv2.circle(image,(i[0],i[1]),4,(255,255,0),-1)
  #cv2.circle(img2,(i[0],i[1]),4,(255,255,0),-1) 
cv2.startWindowThread()
cv2.namedWindow('img')
cv2.imshow('img', np.hstack([image,img2])) 

(200, 200, 3)
0
[[  0.83692331  -0.11023329  25.21875233]
 [  0.11023329   0.83692331 -11.97607513]]
(67, 2)
139
[67 62  9  7] 67
